# Tasks

In [84]:
# libraries needed
import os
import tempfile
import requests
import math
import itertools

### Task 1: Binary Representations

Create the following functions in Python, demonstrating their use with examples and tests.

The function rotl(x, n=1) that rotates the bits in a 32-bit unsigned integer to the left n places.

The function rotr(x, n=1) that rotates the bits in a 32-bit unsigned integer to the right n places.

The function ch(x, y, z) that chooses the bits from y where x has bits set to 1 and bits in z where x has bits set to 0.

The function maj(x, y, z) which takes a majority vote of the bits in x, y, and z.
The output should have a 1 in bit position i where at least two of x, y, and z have 1's in position i.
All other output bit positions should be 0.

In [20]:
# ROTATE LEFT
def rotl(x, n=1):
    """Rotate left (circular shift) a 32-bit unsigned integer x by n bits."""
    if n == 0: return 0
    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

0

In [2]:
# ROTATE RIGHT
def rotr(x, n=1):
    """Rotate right (circular shift) a 32-bit unsigned integer x by n bits."""
    if n == 0: return 0
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

In [3]:
# CHOOSE FUNCTION
def ch(x, y, z):
    """
    For each bit position:
    - Choose bit from y if bit in x is 1
    - Choose bit from z if bit in x is 0
    """
    return (x & y) ^ (~x & z) & 0xFFFFFFFF

In [4]:
# MAJORITY FUNCTION
def maj(x, y, z):
    """
    For each bit position, return 1 if at least two of the bits in x, y, z are 1.
    """
    return ((x & y) ^ (x & z) ^ (y & z)) & 0xFFFFFFFF

In [26]:
# Choice Function
def ch(x, y, z):
    """
    For each bit position:
    - Choose bit from y if bit in x is 1
    - Choose bit from z if bit in x is 0
    """
    return (x & y) ^ (~x & z) & 0xFFFFFFFF

In [79]:
# Tests for Task 1
def test_rotl():
    assert rotl(0x00000001, 1) == 0x00000002
    assert rotl(0x80000000, 1) == 0x00000001
    assert rotl(0x12345678, 4) == 0x23456781
    assert rotl(0xFFFFFFFF, 16) == 0xFFFFFFFF

def test_rotr():
    assert rotr(0x00000001, 1) == 0x80000000
    assert rotr(0x80000000, 1) == 0x40000000
    assert rotr(0x12345678, 4) == 0x81234567
    assert rotr(0xFFFFFFFF, 16) == 0xFFFFFFFF
    

# run tests
test_rotl()
test_rotr()

### Task 2: Hash Functions

The following hash function is from The C Programming Language by Brian Kernighan and Dennis Ritchie.
Convert it to Python, test it, and suggest why the values 31 and 101 are used.



``unsigned hash(char *s) {``

``    unsigned hashval;``

``    for (hashval = 0; *s != '\0'; s++)``

``        hashval = *s + 31 * hashval;``

``    return hashval % 101;``

``}``

In [27]:
def hash_function(text) -> int:
    '''
    
    Args:
    Returns:
    '''
    hashval = 0
    
    #for each char in text provided
    for char in text:
        # add the ascii value of the current character & the current hashval * 31
        hashval = ord(char) + (31 * hashval)
    # after processing all characters, get the remainder of the result after dividing by 101
    return hashval % 101

# Example usage
test_strings = ["hello", "world", "python", "hash", "function", "hello world"]



for string in test_strings:
    hash_value = hash_function(string)
    print(f"{string}| {hash_value}")
    
# Demonstrate collision
print("\nLooking for hash collisions...")
hash_table = {}
for i in range(1000):
    test_string = f"test{i}"
    hash_value = hash_function(test_string)
    
    if hash_value in hash_table:
        print(f"Collision found! Both '{test_string}' and '{hash_table[hash_value]}' hash to {hash_value}")
        break
    
    hash_table[hash_value] = test_string
else:
    print("No collisions found in first 1000 test strings")


hello| 17
world| 34
python| 91
hash| 15
function| 100
hello world| 13


### Task 3: SHA256

Write a Python function that calculates the SHA256 padding for a given file.
The function should take a file path as input.
It should print, in hex, the padding that would be applied to it.
The specification states that the following should be appended to a message:

a1 bit;
enough 0 bits so the length in bits of padded message is the smallest possible multiple of 512;
the length in bits of the original input as a big-endian 64-bit unsigned integer.
The example in the specification is a file containing the three bytes abc:

`01100001 01100010 01100011`

The output would be:

`80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 18`

In [55]:
def print_padding_hex(file_path) -> None:
    """
    Print the SHA-256 padding for a file in hex format.
    
    Args:
        file_path: Path to the file
    """
    
    def calculate_sha256_padding(file_path) -> bytes:
        """
        Nested function to calculate the SHA-256 padding that would be applied to a file.

        Args:
            file_path: Path to the file 

        Returns:
            bytes: The padding bytes that would be added to the file
        """
        # Get the file size in bits
        file_size_bytes = os.path.getsize(file_path)
        file_size_bits = file_size_bytes * 8

        # Start with the '1' bit followed by zeros (0x80 is 10000000 in binary)
        padding = bytearray([0x80])

        # Calculate how many zero bytes we need
        # We need to make the message length (in bits) congruent to 448 modulo 512
        # This formula gives us the number of pad bytes needed (including the 0x80 byte)
        # We adjust by subtracting 1 since we already added the first padding byte (0x80)
        padding_bytes_needed = ((56 - (file_size_bytes + 1) % 64) % 64) 

        # Add the zero bytes 
        padding.extend([0x00] * padding_bytes_needed)

        # Add the original message length as a 64-bit big-endian integer
        # We're creating 8 bytes (64 bits) representing the file size in bits
        for i in range(8):
            # Shift right by multiples of 8 and mask to get each byte
            # Starting from most significant byte (big-endian)
            padding.append((file_size_bits >> (56 - i * 8)) & 0xFF)

        return bytes(padding)
    
    padding = calculate_sha256_padding(file_path)
    
    # Print in hex format with line breaks (similar to the example)
    hex_padding = ' '.join([f'{b:02x}' for b in padding])
    
    # Break every 26 bytes (52 chars plus spaces)
    formatted_padding = ''
    for i in range(0, len(hex_padding), 78):
        formatted_padding += hex_padding[i:i+78] + '\n'
    
    print(formatted_padding)

In [45]:
# Test with a file containing "abc" 


def test_print_padding_hex() -> None:
    """
    Test the SHA-256 padding functionality with the string 'abc'
    """
    # Create a test file with 'abc'
    with open("test.txt", "wb") as f:
        f.write(b"abc")
    
    print("SHA-256 padding for 'abc':")
    print_padding_hex("test.txt")
    
    # Clean up the test file
    os.remove("test.txt")
    
    # Expected output for reference (3-byte file):
    # - First byte should be 0x80 (the '1' bit followed by zeros)
    # - Followed by 52 zero bytes
    # - Last 8 bytes should represent 24 bits (3 bytes * 8): 0x0000000000000018
    
    print("\nTest completed. Verify that the output shows:")
    print("1. The first byte is '80'")
    print("2. The last 8 bytes are '00 00 00 00 00 00 00 18'")
    print("3. Total padding length is 61 bytes (1 + 52 + 8)")
test_print_padding_hex()


SHA-256 padding for 'abc':
80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 
00 00 00 00 00 00 00 00 18


Test completed. Verify that the output shows:
1. The first byte is '80'
2. The last 8 bytes are '00 00 00 00 00 00 00 18'
3. Total padding length is 61 bytes (1 + 52 + 8)


### Task 4: Prime Numbers

Calculate the first 100 prime numbers using two different algorithms.
Any algorithms that are well-established and works correctly are okay to use.
Explain how the algorithms work.

**Algorithm One: Sieve of Eratosthenes** 
The Sieve of Eratosthenes is an ancient and efficient algorithm for finding all prime numbers up to a specified limit. It works by iteratively marking the multiples of each prime number as composite (not prime).

In [56]:
def sieve_of_eratosthenes(n) -> list:
    """
    Find the first 100 prime numbers using the Sieve of Eratosthenes algorithm.
    
    This function implements the classical Sieve of Eratosthenes algorithm to efficiently
    find prime numbers.
    
    Args:
        n (int):The upper limit to search for primes. Should be large enough to 
                contain at least 100 prime numbers [600 works for me].
    
    Returns:
        list: A list containing the first 100 prime numbers.
    """
    # Create a bool array where all entries are initially True
    # A value in prime[i] will finally be False if i is not a prime, else True
    prime = [True for i in range(n+1)]
    p = 2
    
    # Start from the first prime number, 2
    while p * p <= n:
        # If prime[p] is True, then it's a prime
        if prime[p] == True:
            # Update all multiples of p
            for i in range(p * p, n+1, p):
                prime[i] = False
        p += 1
    
    # Create a list of all prime numbers
    primes = []
    for p in range(2, n+1):
        if prime[p]:
            primes.append(p)
            if len(primes) == 100:
                break
    
    return primes

# Get the first 100 prime numbers
primes = sieve_of_eratosthenes(600)  # 600 is sufficient to find the first 100 primes
print(primes)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


**Algorithm 2: Incremental Sieve**

The Incremental Sieve uses a dictionary to track composite numbers and their smallest prime factors:

It processes numbers one by one, starting from 2
If a number is not in the dictionary, it's prime
When we find a prime, we add its square to the composites dictionary
For composite numbers, we advance the smallest prime factor to the next multiple
We continue until we've found the desired number of primes

In [49]:
def incremental_sieve(n) -> list:
    """
    Find the first n prime numbers using an incremental sieve approach.
    
    Args:
        n: Number of primes to find
        
    Returns:
        A list of the first n prime numbers
    """
    primes = []
    # Dictionary to store smallest prime factor for each composite number
    composites = {}
    
    num = 2  # Start with first prime
    
    while len(primes) < n:
        # If num is not in composites, it's prime
        if num not in composites:
            primes.append(num)
            # Mark its first multiple (num*num) as composite
            composites[num * num] = num
        else:
            # num is composite, find next multiple of its smallest prime factor
            prime_factor = composites.pop(num)
            next_composite = num + prime_factor
            
            # Make sure we don't overwrite an existing entry
            while next_composite in composites:
                next_composite += prime_factor
                
            composites[next_composite] = prime_factor
            
        num += 1
        
    return primes

print(incremental_sieve(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


In [81]:
# Testing algorithms against one another
assert(( sieve_of_eratosthenes(600)) == incremental_sieve(100))

### Task 5: Roots

Calculate the first **32 bits** of the fractional part of the square roots of the first 100 prime numbers.

To avhieve this task we must use the function from task number 4 to get the first 100 prime numbers, and then calculate the first 32 bits of the fractional part of the square roots of them.

In [80]:
first_100_prime_numbers = incremental_sieve(100)

# get the first 100 prime numbers ( from previous task )
print(first_100_prime_numbers)

def roots(primes) -> None:

    # calculate the square root of each prime number
    for prime in primes:

        #Calculate the square root
        sqrt_value = math.sqrt(prime)
    
roots(first_100_prime_numbers)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]


### Task 6: Proof of work

In [ ]:
# Proof of work component


### Task 7: Turing Machines

Design a Turing Machine that adds 1 to a binary number on its tape.
The machine should start at the left-most non-blank symbol.
It should treat the right-most symbol as the least significant bit.

For example, suppose the following is on the tape at the start:

``100111``

Your Turing machine should leave the following on the tape when it completes:

``101000``

In [82]:
# create Tape class
class Tape(object):
    
    blank_symbol = " "
    
    def __init__(self,
                 tape_string = ""):
        self.__tape = dict((enumerate(tape_string)))
        
    def __str__(self):
        s = ""
        min_used_index = min(self.__tape.keys()) 
        max_used_index = max(self.__tape.keys())
        for i in range(min_used_index, max_used_index):
            s += self.__tape[i]
        return s    
   
    def __getitem__(self,index):
        if index in self.__tape:
            return self.__tape[index]
        else:
            return Tape.blank_symbol

    def __setitem__(self, pos, char):
        self.__tape[pos] = char 


# Create TuringMachine Class
class TuringMachine(object):
    
    def __init__(self, 
                 tape = "", 
                 blank_symbol = " ",
                 initial_state = "",
                 final_states = None,
                 transition_function = None):
        self.__tape = Tape(tape)
        self.__head_position = 0
        self.__blank_symbol = blank_symbol
        self.__current_state = initial_state
        if transition_function == None:
            self.__transition_function = {}
        else:
            self.__transition_function = transition_function
        if final_states == None:
            self.__final_states = set()
        else:
            self.__final_states = set(final_states)
        
    def get_tape(self): 
        return str(self.__tape)
    
    def step(self):
        char_under_head = self.__tape[self.__head_position]
        x = (self.__current_state, char_under_head)
        if x in self.__transition_function:
            y = self.__transition_function[x]
            self.__tape[self.__head_position] = y[1]
            if y[2] == "R":
                self.__head_position += 1
            elif y[2] == "L":
                self.__head_position -= 1
            self.__current_state = y[0]

    def final(self):
        if self.__current_state in self.__final_states:
            return True
        else:
            return False

In [78]:
# Test

initial_state = "init",
accepting_states = ["final"],
transition_function = {("init","0"):("init", "1", "R"),
                       ("init","1"):("init", "0", "R"),
                       ("init"," "):("final"," ", "N"),
                       }
final_states = {"final"}

t = TuringMachine("010011001 ",
                  initial_state = "init",
                  final_states = final_states,
                  transition_function=transition_function)

print("Input on Tape:\n" + t.get_tape())

while not t.final():
    t.step()

print("Result of the Turing machine calculation:")
print(t.get_tape())


Input on Tape:
010011001
Result of the Turing machine calculation:
101100110


### Task 8: Computational Complexity

Implement bubble sort in Python, modifying it to count the number of comparisons made during sorting.
Use this function to sort all permutations of the list:

``L = [1, 2, 3, 4, 5]``
For each permutation, print the permutation itself followed by the number of comparisons required to sort it.

In [85]:
def bubble_sort_with_comparison_count(arr):
    n = len(arr)
    comparisons = 0
    arr = arr.copy()  # To avoid modifying the original permutation
    for i in range(n):
        for j in range(0, n-i-1):
            comparisons += 1
            if arr[j] > arr[j+1]:
                arr[j], arr[j+1] = arr[j+1], arr[j]
    return comparisons

L = [1, 2, 3, 4, 5]
perms = itertools.permutations(L)

for perm in perms:
    comp_count = bubble_sort_with_comparison_count(list(perm))
    print(f"{perm} -> Comparisons: {comp_count}")


(1, 2, 3, 4, 5) -> Comparisons: 10
(1, 2, 3, 5, 4) -> Comparisons: 10
(1, 2, 4, 3, 5) -> Comparisons: 10
(1, 2, 4, 5, 3) -> Comparisons: 10
(1, 2, 5, 3, 4) -> Comparisons: 10
(1, 2, 5, 4, 3) -> Comparisons: 10
(1, 3, 2, 4, 5) -> Comparisons: 10
(1, 3, 2, 5, 4) -> Comparisons: 10
(1, 3, 4, 2, 5) -> Comparisons: 10
(1, 3, 4, 5, 2) -> Comparisons: 10
(1, 3, 5, 2, 4) -> Comparisons: 10
(1, 3, 5, 4, 2) -> Comparisons: 10
(1, 4, 2, 3, 5) -> Comparisons: 10
(1, 4, 2, 5, 3) -> Comparisons: 10
(1, 4, 3, 2, 5) -> Comparisons: 10
(1, 4, 3, 5, 2) -> Comparisons: 10
(1, 4, 5, 2, 3) -> Comparisons: 10
(1, 4, 5, 3, 2) -> Comparisons: 10
(1, 5, 2, 3, 4) -> Comparisons: 10
(1, 5, 2, 4, 3) -> Comparisons: 10
(1, 5, 3, 2, 4) -> Comparisons: 10
(1, 5, 3, 4, 2) -> Comparisons: 10
(1, 5, 4, 2, 3) -> Comparisons: 10
(1, 5, 4, 3, 2) -> Comparisons: 10
(2, 1, 3, 4, 5) -> Comparisons: 10
(2, 1, 3, 5, 4) -> Comparisons: 10
(2, 1, 4, 3, 5) -> Comparisons: 10
(2, 1, 4, 5, 3) -> Comparisons: 10
(2, 1, 5, 3, 4) -> C